In [1]:
#     DataQuest Project - Profitable App Profiles for the App Store
#                       and Google Play Markets

# We will be considering 2 datasets in this project: 
# The app store and google play apps datasets. The goal is to find an optimal 
# Category for an app developer to build a free app in, based on the factor
# columns provided by the data.

#The project does not require any external libraries other than csv.

In [2]:
from csv import reader #Import the datasets and read them
opened_file = open('googleplaystore.csv', encoding = "utf8")
read_file = reader(opened_file)
apps_data_google = list(read_file)

opened_file = open('AppleStore.csv', encoding = "utf8")
read_file = reader(opened_file)
apps_data_apple = list(read_file)


In [3]:
apps_data_google[0] #The first row contains column names

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

In [4]:
apps_data_apple[0]

['',
 'id',
 'track_name',
 'size_bytes',
 'currency',
 'price',
 'rating_count_tot',
 'rating_count_ver',
 'user_rating',
 'user_rating_ver',
 'ver',
 'cont_rating',
 'prime_genre',
 'sup_devices.num',
 'ipadSc_urls.num',
 'lang.num',
 'vpp_lic']

In [5]:
len(apps_data_apple) #length of the apple dataset

7198

In [6]:
len(apps_data_google)

10842

In [7]:
#Data cleaning
# 1. remove non-English apps, duplicates
# 2. remove or correct inaccurate data
# 3. remove non-free apps as those will not be included in the analysis

In [8]:
# according to https://www.kaggle.com/lava18/google-play-store-apps/discussion the google play dataset
#has a wrong entry at row 10472
#according to the source, Category is not present and hence column shift has happened in this row

In [9]:
apps_data_google[10473] #inspecting the required entry

['Life Made WI-Fi Touchscreen Photo Frame',
 '1.9',
 '19',
 '3.0M',
 '1,000+',
 'Free',
 '0',
 'Everyone',
 '',
 'February 11, 2018',
 '1.0.19',
 '4.0 and up']

In [10]:
apps_data_google[0] #indeed the category entry is missing, and for example
#the "installs column" has an entry "Free"
#indicating that there was a column shift. 

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

In [11]:
del apps_data_google[10473] #Deleting the entry as not much data is lost here -
#only a single row.

In [12]:
#Checking the source for the apple dataset https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/discussion
#I could not find any reports of wrong data.

In [13]:
#Finding duplicates
#google play
unique_google = []
dup_google = []

for entry in apps_data_google[1:]: #sepparating the app names into duplicates
                                   # and unique apps
    name = entry[0]
    if name in unique_google:
        dup_google.append(name)
    else:
        unique_google.append(name)

#App store
unique_apps = []
dup_apps = []

for entry in apps_data_apple[1:]:
    name = entry[0]
    if name in unique_apps:
        dup_apps.append(name)
    else:
        unique_apps.append(name)
        

In [14]:
dup_apps #no duplicates in the appstore dataset.


[]

In [15]:
len(dup_google) #quite a few duplicates in the google dataset

1181

In [16]:
#Using the 2 simple functions above, we have checked that only the google dataset has missing entries.
print(len(dup_apps))
print(len(dup_google))

0
1181


In [17]:

#Now the duplicates from the dataset will be removed. 
#The removal process is not going to be random and the removal criterion is going to be the number of reviews for the google
#play dataset 
#This is because the version of the app with the largest numbner of reviews is the latest
#version of that app.


In [18]:
#google play
reviews_max = {}
for entry in apps_data_google[1:]:
    name = entry[0]
    n_reviews = float(entry[3])
    if (name in reviews_max and reviews_max[name] <= n_reviews):
        temp = {name: n_reviews}
        reviews_max.update(temp)
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
       
        
        
        

In [22]:
print(len(reviews_max))
#this dictionary contains unique app entries with 
#the maximised numbers of reviews


9659


In [24]:
#now using that dictionary, we can remove the duplicate entries in the google play dataset
google_clean = []
added = []
for entry in apps_data_google[1:]:
    name = entry[0]
    rev = float(entry[3])
    if (name in reviews_max) and (rev == reviews_max[name]) and (name not in added):
        google_clean.append(entry)
        added.append(name)

In [25]:
len(google_clean) #since the number of entries of the clean dataset matches the number of entries in the dictionary, we conclude
#that the dataset indeed contains no duplicates.

9659

In [26]:
#The second stage of the cleaning process is to remove apps that are not targeted at the English-speaking audience.
#This can be done through filtering the symbols according to the ASCII system. 
#(range of symbols 0-127.)

In [27]:
#The following function takes in a string, analyses it char by char, and determines if its order is in the required range.
#If a single char is out of that range, it very likely that the app has a foreign char in its name and is therefore not 
#targeted at an English-speaking audience.
#The function returns a bool with True meaning the string is in English and False if at least 3 chars are out of range.
#This is so that an app with up to emojis is not filtered out. 
def ASCII_filter(string):
    count = 0
    for char in string:
        if (ord(char) > 127):
            count +=1
    
    if count > 3:
        return False
    else:
        return True
            
            

        


In [28]:
#testing 
ASCII_filter("Vladimir Sapozhnikov")


True

In [29]:
ASCII_filter("Vl😁dimir")

True

In [30]:
ASCII_filter("Владимир Сапожников") 

False

In [31]:
#The function works correctly and can therefore be applied to filter out the undesired entries.

In [32]:
google_play_clean = []
for entry in google_clean:
    name = entry[0]
    if ASCII_filter(name) == True:
        google_play_clean.append(entry)

In [33]:
len(google_play_clean)


9614

In [34]:
#Now the apple dataset
appstore_clean = []
for entry in apps_data_apple:
    name = entry[3]
    if ASCII_filter(name) == True:
        appstore_clean.append(entry)
        

In [35]:
len(appstore_clean)

7198

In [36]:
#Double check
count = 0
for entry in appstore_clean:
    name = entry[3]
    if ASCII_filter(name) == False:
        count +=1
        
count        

0

In [37]:
count = 0
for entry in google_play_clean:
    name = entry[0]
    if ASCII_filter(name) == False:
        count +=1
        
count          

0

In [38]:
#Since the count is 0 in both datasets, the datasets are likely to match the language requirement.
#(we cannot be 100% sure though, as the filter might miss names that are less than or 3 characters long and are foreign.)
#The function could be further improved by looking at the ratio of foreign chars to english chars rather than an arbitrarily
#chosen number.

In [39]:
#The last part of the cleaning process is to remove all the paid apps from both datasets.
#With the google play dataset, we simply leave the apps that say "Free" in the Type column.
#With the app store dataset, we only leave tha apps that have a price of "0.0"
google_play_final = []
app_store_final = []

for entry in google_play_clean[1:]:
    isfree = entry[6]
    if isfree == "Free" or isfree == "free":
        google_play_final.append(entry)

for entry in appstore_clean[1:]:
    price = entry[7]
    if price == "0.0" or price == "0" or price == "$0.0" or price == "$0":
        app_store_final.append(entry)
    

In [40]:
len(google_play_final)

8862

In [41]:
len(app_store_final)

1443

In [42]:
apps_data_apple[0]

['',
 'id',
 'track_name',
 'size_bytes',
 'currency',
 'price',
 'rating_count_tot',
 'rating_count_ver',
 'user_rating',
 'user_rating_ver',
 'ver',
 'cont_rating',
 'prime_genre',
 'sup_devices.num',
 'ipadSc_urls.num',
 'lang.num',
 'vpp_lic']

In [43]:
#adding back the headers
google_play_final.insert(0, apps_data_google[0])
app_store_final.insert(0, apps_data_apple[0])


In [44]:
google_play_final[:20]

[['App',
  'Category',
  'Rating',
  'Reviews',
  'Size',
  'Installs',
  'Type',
  'Price',
  'Content Rating',
  'Genres',
  'Last Updated',
  'Current Ver',
  'Android Ver'],
 ['U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  'ART_AND_DESIGN',
  '4.7',
  '87510',
  '8.7M',
  '5,000,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'August 1, 2018',
  '1.2.4',
  '4.0.3 and up'],
 ['Sketch - Draw & Paint',
  'ART_AND_DESIGN',
  '4.5',
  '215644',
  '25M',
  '50,000,000+',
  'Free',
  '0',
  'Teen',
  'Art & Design',
  'June 8, 2018',
  'Varies with device',
  '4.2 and up'],
 ['Pixel Draw - Number Art Coloring Book',
  'ART_AND_DESIGN',
  '4.3',
  '967',
  '2.8M',
  '100,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design;Creativity',
  'June 20, 2018',
  '1.1',
  '4.4 and up'],
 ['Paper flowers instructions',
  'ART_AND_DESIGN',
  '4.4',
  '167',
  '5.6M',
  '50,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'March 26, 2017',
  '1.0',
  '2.3 and up'],
 ['Sm

In [45]:

#looking at Category, Genres and Installs for Google
#prime genre and sup_devices.num

In [46]:
#Frequecny table
#identify unique genres and categories for google data
genre_freq = {}
category_freq = {}

for entry in google_play_final[1:]:
    genre = entry[9]
    if genre in genre_freq:
        genre_freq[genre] +=1
    else:
        genre_freq[genre] = 1
        
for entry in google_play_final[1:]:
    category = entry[1]
    if category in category_freq:
        category_freq[category] +=1
    else:
        category_freq[category] = 1
        

#frequncies in terms of proportions and percentages here

genre_freq_prop = {}
genre_freq_per = {}

cat_freq_prop = {}
cat_freq_per = {}

total_apps = len(google_play_final)

for key in genre_freq:
    prop = (genre_freq[key]/total_apps)
    per = prop*100
    genre_freq_prop[key] = prop
    genre_freq_per[key] = per
    
for key in category_freq:
    prop = (category_freq[key]/total_apps)
    per = prop*100
    cat_freq_prop[key] = prop
    cat_freq_per[key] = per
    



In [47]:
#Now lets sort them

In [48]:
def freq_table_as_tuple_table(freq_table):
    tuple_table = []
    for key in freq_table:
        key_val_as_tuple = (freq_table[key], key)
        tuple_table.append(key_val_as_tuple)
    return tuple_table    

sorted_genre_freq = sorted(freq_table_as_tuple_table(genre_freq), reverse = True)
sorted_genre_prop = sorted(freq_table_as_tuple_table(genre_freq_prop), reverse = True)
sorted_genre_per = sorted(freq_table_as_tuple_table(genre_freq_per), reverse = True)

sorted_cat_freq = sorted(freq_table_as_tuple_table(category_freq), reverse = True)
sorted_cat_prop = sorted(freq_table_as_tuple_table(cat_freq_prop), reverse = True)
sorted_cat_per = sorted(freq_table_as_tuple_table(cat_freq_per), reverse = True)


In [49]:
#now do the same with the appstore dataset


In [50]:
prime_genre_freq = {}

for entry in app_store_final[1:]:
    prime = entry[12]
    if prime in prime_genre_freq:
        prime_genre_freq[prime] +=1
    else:
        prime_genre_freq[prime] = 1
        


#frequncies in terms of proportions and percentages here
prime_genre_prop = {}
prime_genre_per = {}

total_apps = len(app_store_final)

for key in prime_genre_freq:
    prop = (prime_genre_freq[key]/total_apps)
    per = prop*100
    prime_genre_prop[key] = prop
    prime_genre_per[key] = per
    



In [51]:
def freq_table_as_tuple_table(freq_table):
    tuple_table = []
    for key in freq_table:
        key_val_as_tuple = (freq_table[key], key)
        tuple_table.append(key_val_as_tuple)
    return tuple_table    

sorted_prime_freq = sorted(freq_table_as_tuple_table(prime_genre_freq), reverse = True)
sorted_prime_prop = sorted(freq_table_as_tuple_table(prime_genre_prop), reverse = True)
sorted_prime_per = sorted(freq_table_as_tuple_table(prime_genre_per), reverse = True)


In [52]:
#There is a lot of similar and repeated code here.
#Of course this could have been done by creating a general function that would do this for any dictionary we 
#input, but I decided not to go with that approach as this could affect readability by making the code too abstract and harder
#to interpret.

In [53]:
#Lets check out the results

In [54]:
#App Store prime genre frequency table expressed as a percentage of the total number of apps

print(sorted_prime_per)

[(41.41274238227147, 'Games'), (7.894736842105263, 'Education'), (7.825484764542937, 'Entertainment'), (4.016620498614958, 'Finance'), (3.9473684210526314, 'Book'), (3.8781163434903045, 'Social Networking'), (3.8088642659279777, 'Photo & Video'), (3.6703601108033244, 'Lifestyle'), (3.6011080332409975, 'Utilities'), (2.9778393351800556, 'Shopping'), (2.5623268698060944, 'Health & Fitness'), (2.146814404432133, 'Sports'), (2.008310249307479, 'News'), (1.7313019390581719, 'Travel'), (1.5927977839335181, 'Food & Drink'), (1.3850415512465373, 'Navigation'), (1.2465373961218837, 'Productivity'), (1.0387811634349031, 'Reference'), (0.9695290858725761, 'Weather'), (0.8310249307479225, 'Business'), (0.7617728531855956, 'Music'), (0.3462603878116343, 'Catalogs'), (0.2770083102493075, 'Medical')]


In [55]:
#Google play genre frequency table expressed as a percentage of the total number of apps
print(sorted_genre_per) 

[(8.450863138892023, 'Tools'), (6.070179397495204, 'Entertainment'), (5.348076272142616, 'Education'), (4.592124562789123, 'Business'), (3.8925871601038025, 'Productivity'), (3.8925871601038025, 'Lifestyle'), (3.7007785174320205, 'Finance'), (3.5315355974275078, 'Medical'), (3.463838429425702, 'Sports'), (3.317161232088458, 'Personalization'), (3.2381812027530184, 'Communication'), (3.102786866749408, 'Action'), (3.0802211440821394, 'Health & Fitness'), (2.944826808078529, 'Photography'), (2.798149610741284, 'News & Magazines'), (2.6627552747376737, 'Social'), (2.324269434728647, 'Travel & Local'), (2.245289405393208, 'Shopping'), (2.1437436533904997, 'Books & Reference'), (2.042197901387792, 'Simulation'), (1.8616721200496444, 'Dating'), (1.8503892587160102, 'Arcade'), (1.771409229380571, 'Video Players & Editors'), (1.7601263680469368, 'Casual'), (1.399074805370642, 'Maps & Navigation'), (1.241114746699763, 'Food & Drink'), (1.128286133363421, 'Puzzle'), (0.9928917973598104, 'Racing'

In [56]:
#Google play category frequency table expressed as a percentage of the total number of apps
print(sorted_cat_per)

[(18.898792733837304, 'FAMILY'), (9.725826469592688, 'GAME'), (8.462146000225657, 'TOOLS'), (4.592124562789123, 'BUSINESS'), (3.9038700214374367, 'LIFESTYLE'), (3.8925871601038025, 'PRODUCTIVITY'), (3.7007785174320205, 'FINANCE'), (3.5315355974275078, 'MEDICAL'), (3.396141261423897, 'SPORTS'), (3.317161232088458, 'PERSONALIZATION'), (3.2381812027530184, 'COMMUNICATION'), (3.0802211440821394, 'HEALTH_AND_FITNESS'), (2.944826808078529, 'PHOTOGRAPHY'), (2.798149610741284, 'NEWS_AND_MAGAZINES'), (2.6627552747376737, 'SOCIAL'), (2.335552296062281, 'TRAVEL_AND_LOCAL'), (2.245289405393208, 'SHOPPING'), (2.1437436533904997, 'BOOKS_AND_REFERENCE'), (1.8616721200496444, 'DATING'), (1.7939749520478394, 'VIDEO_PLAYERS'), (1.399074805370642, 'MAPS_AND_NAVIGATION'), (1.241114746699763, 'FOOD_AND_DRINK'), (1.1621347173643235, 'EDUCATION'), (0.9590432133589079, 'ENTERTAINMENT'), (0.9364774906916393, 'LIBRARIES_AND_DEMO'), (0.9251946293580051, 'AUTO_AND_VEHICLES'), (0.8236488773552973, 'HOUSE_AND_HOME'

In [57]:
#Lets briefly analyse the frequency tables

In [58]:
#Google play

#The top 3 most popular genres in google play are Tools, Entertainment and Education but due to the large number of 
#genres, they only hold  rouhly 8.5, 6 and 5 percent of the market respectively.

#looking at categories for Google play, we see Family holding roughly 19 percent of the market, Games holding roughly 10,
#and Tools roughly 8.5

#The general impression is that there is a good mix of practical apps and apps designed for entertainment 

#App Store
#The largest Category here is Games, holding roughly 41.5 percent of the market. It is follwed by education with around 8 percent
#and Entertainment holding approximately the same percentage.

#In App Store the skew is heavily towards games, and seeing how google play has a good mix of apps designed for the entertainment
#purposes maybe it would be a good idea to develop a game (Action and Simulation being the most popular in google play).
#This of course is speculation at this level, since we have not yet
#looked at the number of downloads and ratings and reviews of the apps, and without those, all we can tell is what the majority
#of apps are, but not their popularity.

#Of course the assumption here would be that most apps are designed to satisfy the largest demand.

In [59]:
#lets look at an average number of installs for each genre in google play.
#Since this information is missing in the app store dataset, we will have to instead consider the average number of reviews

In [60]:
#App store

av_num_ratings = {}

for unique in prime_genre_freq:
    sum_ratings = 0
    sum_apps = 0
    for entry in app_store_final[1:]:
        genre_app = entry[12]
        if genre_app == unique:
            sum_ratings += float(entry[6]) # the number of ratings here added to sum ratings
            sum_apps +=1
            
    av_num_ratings[unique] = (sum_ratings/sum_apps)
    
sorted_av_num_ratings = sorted(freq_table_as_tuple_table(av_num_ratings), reverse = True)    

In [61]:
sorted_av_num_ratings #surprising results, but first, lets compute this metric for the google play data

[(10656.236363636364, 'Photo & Video'),
 (1628.906976744186, 'Shopping'),
 (1325.7857142857142, 'Social Networking'),
 (981.92, 'Travel'),
 (820.3333333333334, 'Productivity'),
 (611.7857142857143, 'Weather'),
 (375.7345132743363, 'Entertainment'),
 (315.0689655172414, 'News'),
 (235.10344827586206, 'Finance'),
 (193.67741935483872, 'Sports'),
 (162.14882943143812, 'Games'),
 (153.0, 'Business'),
 (121.65789473684211, 'Education'),
 (82.88461538461539, 'Utilities'),
 (71.90566037735849, 'Lifestyle'),
 (62.0, 'Music'),
 (54.35087719298246, 'Book'),
 (44.32432432432432, 'Health & Fitness'),
 (15.782608695652174, 'Food & Drink'),
 (14.933333333333334, 'Reference'),
 (14.45, 'Navigation'),
 (3.0, 'Medical'),
 (0.0, 'Catalogs')]

In [62]:
#The problem with the installs column is that the entries are not very precise, eg 10000+, 1000000+ and so on.
#We do not require great precision here, so we will treat an app with 100000+ installs as an app with 100000 installs. (at least)

In [63]:
#Google play category

av_num_installs = {}

for cat in category_freq:
    sum_installs = 0
    sum_apps = 0
    for entry in google_play_final[1:]:
        cat_app = entry[1]
        if cat_app == cat:
            sum_installs += float(entry[5].replace('+','').replace(',','')) # stripping away the plusses and comas 
            sum_apps +=1                                                    # to add up the installs
            
    av_num_installs[cat] = (sum_installs/sum_apps)
    
sorted_av_num_installs_cat = sorted(freq_table_as_tuple_table(av_num_installs), reverse = True) 

In [64]:
sorted_av_num_installs_cat

[(38456119.167247385, 'COMMUNICATION'),
 (24727872.452830188, 'VIDEO_PLAYERS'),
 (23253652.127118643, 'SOCIAL'),
 (17840110.40229885, 'PHOTOGRAPHY'),
 (16787331.344927534, 'PRODUCTIVITY'),
 (15588015.603248259, 'GAME'),
 (13984077.710144928, 'TRAVEL_AND_LOCAL'),
 (11640705.88235294, 'ENTERTAINMENT'),
 (10801391.298666667, 'TOOLS'),
 (9549178.467741935, 'NEWS_AND_MAGAZINES'),
 (8767811.894736841, 'BOOKS_AND_REFERENCE'),
 (7036877.311557789, 'SHOPPING'),
 (5201482.6122448975, 'PERSONALIZATION'),
 (5074486.197183099, 'WEATHER'),
 (4188821.9853479853, 'HEALTH_AND_FITNESS'),
 (4056941.7741935486, 'MAPS_AND_NAVIGATION'),
 (3697848.1731343283, 'FAMILY'),
 (3638640.1428571427, 'SPORTS'),
 (2021626.7857142857, 'ART_AND_DESIGN'),
 (1924897.7363636363, 'FOOD_AND_DRINK'),
 (1833495.145631068, 'EDUCATION'),
 (1712290.1474201474, 'BUSINESS'),
 (1437816.2687861272, 'LIFESTYLE'),
 (1387692.475609756, 'FINANCE'),
 (1331540.5616438356, 'HOUSE_AND_HOME'),
 (854028.8303030303, 'DATING'),
 (817657.27272727

In [65]:
# Google play genre
av_num_installs = {}

for genre in genre_freq:
    sum_installs = 0
    sum_apps = 0
    for entry in google_play_final[1:]:
        genre_app = entry[9]
        if genre_app == genre:
            sum_installs += float(entry[5].replace('+','').replace(',','')) # stripping away the plusses and comas 
            sum_apps +=1                                                    # to add up the installs
            
    av_num_installs[genre] = (sum_installs/sum_apps)
    
sorted_av_num_installs_genre = sorted(freq_table_as_tuple_table(av_num_installs), reverse = True) 
sorted_av_num_installs_genre

[(38456119.167247385, 'Communication'),
 (35333333.333333336, 'Adventure;Action & Adventure'),
 (24947335.796178345, 'Video Players & Editors'),
 (23253652.127118643, 'Social'),
 (22888365.48780488, 'Arcade'),
 (19569221.602564104, 'Casual'),
 (18366666.666666668, 'Puzzle;Action & Adventure'),
 (17840110.40229885, 'Photography'),
 (17016666.666666668, 'Educational;Action & Adventure'),
 (16787331.344927534, 'Productivity'),
 (15910645.681818182, 'Racing'),
 (14051476.145631067, 'Travel & Local'),
 (12916666.666666666, 'Casual;Action & Adventure'),
 (12603588.872727273, 'Action'),
 (11339901.3125, 'Strategy'),
 (10802461.246995995, 'Tools'),
 (10000000.0, 'Tools;Education'),
 (10000000.0, 'Role Playing;Brain Games'),
 (10000000.0, 'Lifestyle;Pretend Play'),
 (10000000.0, 'Casual;Music & Video'),
 (10000000.0, 'Card;Action & Adventure'),
 (10000000.0, 'Adventure;Education'),
 (9549178.467741935, 'News & Magazines'),
 (9445583.333333334, 'Music'),
 (9375000.0, 'Educational;Pretend Play'),

In [66]:
#Contrary to the inital hypothesis we have that the most popular category of apps in google play is 'Communication'
#despite this category not even making it into the top 10 of our frequency table for the total number of apps developed.
#So it might be a good idea to develop an app in the 'Communication' category for the google play.

#Looking at the similar metric for the app store, we see that the top 3 most popular categories (largest numbers of reviews)
#are 'Photo & Video', 
# 'Shopping' and 'Social Networking'. From here we might deduce that an app developed for communication could essentially be
# a social networking app, and sicne those seem to be the most popular and have a relatively smaller number of competitors
#(other categories have more apps developed) we could conclude with recommending the developer creates a social networking app.

In [67]:
# Dataquest Project - Vladimir Sapozhnikov 26/06/21